In [1]:
import numpy as np
import pandas as pd

In [ ]:
#first, correct cancer files read count biases

In [2]:
dilution_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name='train')
dilution_calc

,healthy_file_name,healthy_read_counts,cancer_file_label,cancer_file_name,original,cancer_read_counts,correction_factor,corrected read counts
0,WHC556,2061558900,CRC-1,1014_180816,0.455,2224638397,0.926694,2.061559e+09
1,WHC556,2061558900,CRC-2,1531_160616,0.858,2178637561,0.946261,2.061559e+09
2,WHC556,2061558900,CRC-3,1531_180119,0.490,2364417111,0.871910,2.061559e+09
3,WHC556,2061558900,CRC-4,519_210114,0.351,2292445291,0.899284,2.061559e+09
4,WHC556,2061558900,CRC-5,809_030915,0.590,2227096655,0.925671,2.061559e+09
...,...,...,...,...,...,...,...,...
91,healthy total,19726151323,CRC-4,519_210114,0.351,2292445291,8.604852,1.972615e+10
92,healthy total,19726151323,CRC-5,809_030915,0.590,2227096655,8.857340,1.972615e+10
93,healthy total,19726151323,CRC-6,809_110914,0.469,2305429143,8.556390,1.972615e+10
94,healthy total,19726151323,CRC-7,986_100215,0.419,2229529895,8.847673,1.972615e+10


In [3]:
num_list_h = ['556', '557', '558', '559', '560', '561', '562', '563', '564', '565', '566']
num_list_c = ['1014_180816', '1531_160616', '1531_180119', '519_210114', '809_030915', '809_110914', '986_100215', '986_261016']

In [4]:
with open('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\benchmark_region_healthy_may30') as file:
    regions_h = [line.rstrip() for line in file]
len(regions_h)

6

In [5]:
regions_h

['chr1_156028951-156032951',
 'chr19_838960-842960',
 'chr19_35737922-35741922',
 'chr19_41080891-41084891',
 'chr4_185745070-185749070',
 'chr16_75283369-75287369']

In [6]:
regions_c = [region[3:] for region in regions_h]
len(regions_c)

6

In [7]:
regions_c

['1_156028951-156032951',
 '19_838960-842960',
 '19_35737922-35741922',
 '19_41080891-41084891',
 '4_185745070-185749070',
 '16_75283369-75287369']

In [8]:
for num_c in num_list_c:
    for num_h in num_list_h:
        for region in regions_c:
            fn = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\may30-benchmark-raw-cancer-' + \
                 num_c + '-' + region
            df = pd.read_csv(fn, header=None, sep='\t')
            factor = dilution_calc[(dilution_calc.healthy_file_name == 'WHC' + num_h) & \
                              (dilution_calc.cancer_file_name == num_c)]['correction_factor'].values[0]
            df[2] = df[2] * factor
            df.to_csv('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\' + num_c + \
                      '\\WHC' + num_h + '\\benchmark-' + region)

In [7]:
#second, mix!

In [9]:
file_origin_column = []
fraction_column = []
regions_dic = {}
for region_c in regions_c:
    region_h = 'chr' + region_c
    region_column = []
    for num_c in num_list_c:
        for num_h in num_list_h:
            frac_calc = pd.read_excel('C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\dilution_calc.xlsx', sheet_name = num_c)
            fn_c = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\cancer_raw_train\\' + num_c + \
                    '\\WHC' + num_h + '\\benchmark-' + region_c
            fn_h = 'C:\\Users\\mayh\\OneDrive - A STAR\\atac_seq\\model\\healthy_raw_train\\may30-benchmark-raw-healthy-WHC' + \
                    num_h + '-' + region_h
            raw_c = pd.read_csv(fn_c)['2']
            raw_h = pd.read_csv(fn_h, header = None, sep = '\t')[2]
            for fraction in frac_calc['target_fraction']:
                if len(regions_dic.keys()) == 0:
                    file_origin_column.append(num_c + num_h)
                    fraction_column.append(fraction)
                
                healthy_factor = frac_calc[frac_calc.target_fraction == fraction]['healthy_factor'].values[0]
                cancer_factor = frac_calc[frac_calc.target_fraction == fraction]['cancer_factor'].values[0]
                
                raw_mixed = cancer_factor * raw_c + healthy_factor * raw_h
                
                if region_h in ['chr1_156028951-156032951', 'chr19_838960-842960', 'chr19_35737922-35741922']:
                    left_flank_sum = raw_mixed[0:1000].sum()
                    right_flank_sum = raw_mixed[3000:4000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[1850:2050] / flank_mean).mean()
                elif region_h in ['chr4_185745070-185749070', 'chr16_75283369-75287369']:
                    left_flank_sum = raw_mixed[3000:4000].sum()
                    right_flank_sum = raw_mixed[0:1000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[2100:2300] / flank_mean).mean()
                else:    
                    left_flank_sum = raw_mixed[0:1000].sum()
                    right_flank_sum = raw_mixed[3000:4000].sum()
                    flank_mean = (left_flank_sum + right_flank_sum) / 2000
                    avg = (raw_mixed[1700:1900] / flank_mean).mean()
                    
                region_column.append(avg)
    regions_dic[region_c] = region_column

In [10]:
regions_df = pd.DataFrame(regions_dic)
regions_df['fraction'] = fraction_column
regions_df['file_origin'] = file_origin_column
regions_df

,1_156028951-156032951,19_838960-842960,19_35737922-35741922,19_41080891-41084891,4_185745070-185749070,16_75283369-75287369,fraction,file_origin
0,1.201439,1.196565,1.311566,0.246822,0.440862,1.126830,0.000,1014_180816556
1,1.197504,1.199751,1.307552,0.247473,0.440632,1.121444,0.005,1014_180816556
2,1.193548,1.202960,1.303520,0.248125,0.440401,1.116033,0.010,1014_180816556
3,1.189571,1.206192,1.299469,0.248780,0.440170,1.110597,0.015,1014_180816556
4,1.185572,1.209447,1.295399,0.249437,0.439939,1.105137,0.020,1014_180816556
...,...,...,...,...,...,...,...,...
2624,0.809957,1.170645,0.856083,0.375704,0.317986,0.581442,0.350,986_261016566
2625,0.793651,1.164867,0.832208,0.371255,0.326016,0.552423,0.375,986_261016566
2626,0.776727,1.158777,0.807146,0.366715,0.334248,0.521669,0.400,986_261016566
2627,0.759150,1.152348,0.780804,0.362079,0.342690,0.489020,0.425,986_261016566


In [11]:
regions_df.to_csv('train_benchmark.csv')